# Chapter 35: Building a Decentralized Exchange (DEX)

---

Decentralized exchanges (DEXes) are the cornerstone of DeFi, enabling trustless token swaps without intermediaries. In this chapter, we'll build a complete DEX from scratch, inspired by Uniswap V2. We'll create the core smart contracts: a liquidity pool (pair), a factory, and a router. We'll also develop a simple frontend to interact with the exchange, and cover testing and deployment. By the end, you'll have a functional DEX and a deep understanding of how automated market makers (AMMs) work under the hood.

---

## 35.1 Project Overview

### 35.1.1 Requirements and Features

Our DEX will support swapping between any two ERC-20 tokens and providing liquidity to earn fees. Key features:

- **Create a new pair**: Anyone can create a liquidity pool for any two tokens via a factory contract.
- **Add liquidity**: Users can deposit an equivalent value of both tokens to receive LP tokens.
- **Remove liquidity**: Users can burn LP tokens to withdraw their share of the pool.
- **Swap tokens**: Users can swap one token for another, paying a 0.3% fee that goes to liquidity providers.
- **Price oracle**: The pool tracks cumulative prices for use in TWAP oracles (optional).
- **Factory tracking**: The factory keeps a registry of all pairs and allows querying.

### 35.1.2 Architecture Design

We'll follow the Uniswap V2 architecture:

```
┌─────────────────────────────────────────────────────────────┐
│                         Factory                              │
│  • createPair(tokenA, tokenB) → pair address                │
│  • getPair(tokenA, tokenB) → address                        │
│  • allPairs(uint) → address                                 │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                         Pair (per pair)                      │
│  • reserves: reserve0, reserve1                             │
│  • mint(beneficiary) → LP tokens                             │
│  • burn(beneficiary) → underlying tokens                     │
│  • swap(amount0Out, amount1Out, to)                          │
│  • sync()                                                     │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                         Router                               │
│  • addLiquidity(...)                                        │
│  • removeLiquidity(...)                                     │
│  • swapExactTokensForTokens(...)                            │
│  • swapTokensForExactTokens(...)                            │
└─────────────────────────────────────────────────────────────┘
```

**Contracts:**

- **UniswapV2Pair**: The core pool contract, holding reserves and implementing the AMM logic. Inherits from UniswapV2ERC20 (LP token).
- **UniswapV2Factory**: Deploys new pair contracts and keeps a registry.
- **UniswapV2Router**: A helper contract that provides convenient functions for adding/removing liquidity and swapping, handling token approvals and transfers.

We'll also include a simple ERC-20 token for testing.

---

## 35.2 Smart Contract Development

### 35.2.1 Liquidity Pool Contract

The pair contract is the heart of the DEX. It holds reserves of two tokens, issues LP tokens, and implements the swap logic with a constant product formula.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC20/IERC20.sol";
import "@openzeppelin/contracts/utils/math/Math.sol";

contract UniswapV2Pair {
    using Math for uint256;

    // ERC20-like LP token
    string public constant name = "Uniswap V2";
    string public constant symbol = "UNI-V2";
    uint8 public constant decimals = 18;
    uint256 public totalSupply;
    mapping(address => uint256) public balanceOf;
    mapping(address => mapping(address => uint256)) public allowance;

    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);

    function _mint(address to, uint256 amount) internal {
        balanceOf[to] += amount;
        totalSupply += amount;
        emit Transfer(address(0), to, amount);
    }

    function _burn(address from, uint256 amount) internal {
        balanceOf[from] -= amount;
        totalSupply -= amount;
        emit Transfer(from, address(0), amount);
    }

    function _approve(address owner, address spender, uint256 amount) internal {
        allowance[owner][spender] = amount;
        emit Approval(owner, spender, amount);
    }

    function transfer(address to, uint256 amount) external returns (bool) {
        _transfer(msg.sender, to, amount);
        return true;
    }

    function transferFrom(address from, address to, uint256 amount) external returns (bool) {
        if (allowance[from][msg.sender] != type(uint256).max) {
            allowance[from][msg.sender] -= amount;
        }
        _transfer(from, to, amount);
        return true;
    }

    function _transfer(address from, address to, uint256 amount) internal {
        balanceOf[from] -= amount;
        balanceOf[to] += amount;
        emit Transfer(from, to, amount);
    }

    // Pair specific state
    address public factory;
    address public token0;
    address public token1;

    uint112 private reserve0;
    uint112 private reserve1;
    uint32  private blockTimestampLast;

    uint256 public price0CumulativeLast;
    uint256 public price1CumulativeLast;

    event Mint(address indexed sender, uint256 amount0, uint256 amount1);
    event Burn(address indexed sender, uint256 amount0, uint256 amount1, address indexed to);
    event Swap(address indexed sender, uint256 amount0In, uint256 amount1In, uint256 amount0Out, uint256 amount1Out, address indexed to);
    event Sync(uint112 reserve0, uint112 reserve1);

    constructor() {
        factory = msg.sender;
    }

    function initialize(address _token0, address _token1) external {
        require(msg.sender == factory, "UniswapV2: FORBIDDEN");
        token0 = _token0;
        token1 = _token1;
    }

    function getReserves() public view returns (uint112 _reserve0, uint112 _reserve1, uint32 _blockTimestampLast) {
        _reserve0 = reserve0;
        _reserve1 = reserve1;
        _blockTimestampLast = blockTimestampLast;
    }

    function _update(uint256 balance0, uint256 balance1, uint112 _reserve0, uint112 _reserve1) private {
        require(balance0 <= type(uint112).max && balance1 <= type(uint112).max, "UniswapV2: OVERFLOW");
        uint32 blockTimestamp = uint32(block.timestamp % 2**32);
        uint32 timeElapsed = blockTimestamp - blockTimestampLast; // overflow is desired

        if (timeElapsed > 0 && _reserve0 != 0 && _reserve1 != 0) {
            // * never overflows, and + overflow is desired
            price0CumulativeLast += uint256(blockTimestamp - blockTimestampLast) * (uint256(_reserve1) << 112) / _reserve0;
            price1CumulativeLast += uint256(blockTimestamp - blockTimestampLast) * (uint256(_reserve0) << 112) / _reserve1;
        }

        reserve0 = uint112(balance0);
        reserve1 = uint112(balance1);
        blockTimestampLast = blockTimestamp;
        emit Sync(reserve0, reserve1);
    }

    // this low-level function should be called from a contract which performs important safety checks
    // standard uniswap v2 implementation
    function mint(address to) external returns (uint256 liquidity) {
        (uint112 _reserve0, uint112 _reserve1,) = getReserves(); // gas savings
        uint256 balance0 = IERC20(token0).balanceOf(address(this));
        uint256 balance1 = IERC20(token1).balanceOf(address(this));
        uint256 amount0 = balance0 - _reserve0;
        uint256 amount1 = balance1 - _reserve1;

        uint256 _totalSupply = totalSupply;
        if (_totalSupply == 0) {
            liquidity = Math.sqrt(amount0 * amount1) - 1000; // MINIMUM_LIQUIDITY
            _mint(address(0), 1000); // permanently lock the first MINIMUM_LIQUIDITY tokens
        } else {
            liquidity = Math.min(amount0 * _totalSupply / _reserve0, amount1 * _totalSupply / _reserve1);
        }
        require(liquidity > 0, "UniswapV2: INSUFFICIENT_LIQUIDITY_MINTED");
        _mint(to, liquidity);

        _update(balance0, balance1, _reserve0, _reserve1);
        emit Mint(msg.sender, amount0, amount1);
    }

    // this low-level function should be called from a contract which performs important safety checks
    // standard uniswap v2 implementation
    function burn(address to) external returns (uint256 amount0, uint256 amount1) {
        (uint112 _reserve0, uint112 _reserve1,) = getReserves();
        address _token0 = token0;
        address _token1 = token1;
        uint256 balance0 = IERC20(_token0).balanceOf(address(this));
        uint256 balance1 = IERC20(_token1).balanceOf(address(this));
        uint256 liquidity = balanceOf[address(this)];

        uint256 _totalSupply = totalSupply;
        amount0 = liquidity * balance0 / _totalSupply;
        amount1 = liquidity * balance1 / _totalSupply;
        require(amount0 > 0 && amount1 > 0, "UniswapV2: INSUFFICIENT_LIQUIDITY_BURNED");
        _burn(address(this), liquidity);
        _safeTransfer(_token0, to, amount0);
        _safeTransfer(_token1, to, amount1);
        balance0 = IERC20(_token0).balanceOf(address(this));
        balance1 = IERC20(_token1).balanceOf(address(this));

        _update(balance0, balance1, _reserve0, _reserve1);
        emit Burn(msg.sender, amount0, amount1, to);
    }

    // this low-level function should be called from a contract which performs important safety checks
    function swap(uint256 amount0Out, uint256 amount1Out, address to, bytes calldata data) external {
        require(amount0Out > 0 || amount1Out > 0, "UniswapV2: INSUFFICIENT_OUTPUT_AMOUNT");
        (uint112 _reserve0, uint112 _reserve1,) = getReserves();
        require(amount0Out < _reserve0 && amount1Out < _reserve1, "UniswapV2: INSUFFICIENT_LIQUIDITY");

        uint256 balance0;
        uint256 balance1;
        {
            // scope for _token{0,1}, avoids stack too deep errors
            address _token0 = token0;
            address _token1 = token1;
            require(to != _token0 && to != _token1, "UniswapV2: INVALID_TO");
            if (amount0Out > 0) _safeTransfer(_token0, to, amount0Out);
            if (amount1Out > 0) _safeTransfer(_token1, to, amount1Out);
            if (data.length > 0) IUniswapV2Callee(to).uniswapV2Call(msg.sender, amount0Out, amount1Out, data);
            balance0 = IERC20(_token0).balanceOf(address(this));
            balance1 = IERC20(_token1).balanceOf(address(this));
        }
        uint256 amount0In = balance0 > _reserve0 - amount0Out ? balance0 - (_reserve0 - amount0Out) : 0;
        uint256 amount1In = balance1 > _reserve1 - amount1Out ? balance1 - (_reserve1 - amount1Out) : 0;
        require(amount0In > 0 || amount1In > 0, "UniswapV2: INSUFFICIENT_INPUT_AMOUNT");
        {
            // scope for reserve{0,1}Adjusted, avoids stack too deep errors
            uint256 balance0Adjusted = balance0 * 1000 - amount0In * 3;
            uint256 balance1Adjusted = balance1 * 1000 - amount1In * 3;
            require(balance0Adjusted * balance1Adjusted >= uint256(_reserve0) * _reserve1 * 1000**2, "UniswapV2: K");
        }

        _update(balance0, balance1, _reserve0, _reserve1);
        emit Swap(msg.sender, amount0In, amount1In, amount0Out, amount1Out, to);
    }

    // force balances to match reserves
    function skim(address to) external {
        address _token0 = token0;
        address _token1 = token1;
        _safeTransfer(_token0, to, IERC20(_token0).balanceOf(address(this)) - reserve0);
        _safeTransfer(_token1, to, IERC20(_token1).balanceOf(address(this)) - reserve1);
    }

    // force reserves to match balances
    function sync() external {
        _update(IERC20(token0).balanceOf(address(this)), IERC20(token1).balanceOf(address(this)), reserve0, reserve1);
    }

    function _safeTransfer(address token, address to, uint256 value) private {
        (bool success, bytes memory data) = token.call(abi.encodeWithSignature("transfer(address,uint256)", to, value));
        require(success && (data.length == 0 || abi.decode(data, (bool))), "UniswapV2: TRANSFER_FAILED");
    }
}

interface IUniswapV2Callee {
    function uniswapV2Call(address sender, uint256 amount0, uint256 amount1, bytes calldata data) external;
}
```

This contract includes:
- ERC-20 functions for LP tokens.
- `mint` for adding liquidity.
- `burn` for removing liquidity.
- `swap` for swapping tokens.
- `_update` for tracking reserves and price accumulators.
- `skim` and `sync` for edge cases.

**Note:** The `swap` function includes a callback for flash swaps (via `IUniswapV2Callee`), which is an advanced feature.

### 35.2.2 Factory Contract

The factory deploys new pair contracts and keeps a registry.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "./UniswapV2Pair.sol";

contract UniswapV2Factory {
    mapping(address => mapping(address => address)) public getPair;
    address[] public allPairs;

    event PairCreated(address indexed token0, address indexed token1, address pair, uint256);

    function createPair(address tokenA, address tokenB) external returns (address pair) {
        require(tokenA != tokenB, "UniswapV2: IDENTICAL_ADDRESSES");
        (address token0, address token1) = tokenA < tokenB ? (tokenA, tokenB) : (tokenB, tokenA);
        require(token0 != address(0), "UniswapV2: ZERO_ADDRESS");
        require(getPair[token0][token1] == address(0), "UniswapV2: PAIR_EXISTS");

        bytes memory bytecode = type(UniswapV2Pair).creationCode;
        bytes32 salt = keccak256(abi.encodePacked(token0, token1));
        assembly {
            pair := create2(0, add(bytecode, 32), mload(bytecode), salt)
        }
        UniswapV2Pair(pair).initialize(token0, token1);

        getPair[token0][token1] = pair;
        getPair[token1][token0] = pair;
        allPairs.push(pair);
        emit PairCreated(token0, token1, pair, allPairs.length);
    }
}
```

**Key points:**
- Uses `CREATE2` to deploy pairs with deterministic addresses based on token addresses.
- Stores the pair address in a mapping for quick lookup.
- Emits an event for each new pair.

### 35.2.3 Router Contract

The router provides convenient methods for users to interact with pairs. It handles token approvals and transfers.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC20/IERC20.sol";
import "@openzeppelin/contracts/token/ERC20/utils/SafeERC20.sol";
import "./UniswapV2Pair.sol";
import "./UniswapV2Factory.sol";

contract UniswapV2Router {
    using SafeERC20 for IERC20;

    address public immutable factory;

    constructor(address _factory) {
        factory = _factory;
    }

    // **** ADD LIQUIDITY ****
    function addLiquidity(
        address tokenA,
        address tokenB,
        uint256 amountADesired,
        uint256 amountBDesired,
        uint256 amountAMin,
        uint256 amountBMin,
        address to,
        uint256 deadline
    ) external returns (uint256 amountA, uint256 amountB, uint256 liquidity) {
        require(block.timestamp <= deadline, "UniswapV2Router: EXPIRED");
        (amountA, amountB) = _addLiquidity(tokenA, tokenB, amountADesired, amountBDesired, amountAMin, amountBMin);
        address pair = UniswapV2Factory(factory).getPair(tokenA, tokenB);
        require(pair != address(0), "UniswapV2Router: INVALID_PAIR");
        IERC20(tokenA).safeTransferFrom(msg.sender, pair, amountA);
        IERC20(tokenB).safeTransferFrom(msg.sender, pair, amountB);
        liquidity = UniswapV2Pair(pair).mint(to);
    }

    function _addLiquidity(
        address tokenA,
        address tokenB,
        uint256 amountADesired,
        uint256 amountBDesired,
        uint256 amountAMin,
        uint256 amountBMin
    ) internal returns (uint256 amountA, uint256 amountB) {
        address pair = UniswapV2Factory(factory).getPair(tokenA, tokenB);
        if (pair == address(0)) {
            pair = UniswapV2Factory(factory).createPair(tokenA, tokenB);
        }
        (uint112 reserve0, uint112 reserve1,) = UniswapV2Pair(pair).getReserves();
        (address token0,) = tokenA < tokenB ? (tokenA, tokenB) : (tokenB, tokenA);
        if (reserve0 == 0 && reserve1 == 0) {
            (amountA, amountB) = (amountADesired, amountBDesired);
        } else {
            uint256 amountBOptimal = quote(amountADesired, reserve0, reserve1);
            if (amountBOptimal <= amountBDesired) {
                require(amountBOptimal >= amountBMin, "UniswapV2Router: INSUFFICIENT_B_AMOUNT");
                (amountA, amountB) = (amountADesired, amountBOptimal);
            } else {
                uint256 amountAOptimal = quote(amountBDesired, reserve1, reserve0);
                assert(amountAOptimal <= amountADesired);
                require(amountAOptimal >= amountAMin, "UniswapV2Router: INSUFFICIENT_A_AMOUNT");
                (amountA, amountB) = (amountAOptimal, amountBDesired);
            }
        }
    }

    // **** REMOVE LIQUIDITY ****
    function removeLiquidity(
        address tokenA,
        address tokenB,
        uint256 liquidity,
        uint256 amountAMin,
        uint256 amountBMin,
        address to,
        uint256 deadline
    ) external returns (uint256 amountA, uint256 amountB) {
        require(block.timestamp <= deadline, "UniswapV2Router: EXPIRED");
        address pair = UniswapV2Factory(factory).getPair(tokenA, tokenB);
        require(pair != address(0), "UniswapV2Router: INVALID_PAIR");
        IERC20(pair).safeTransferFrom(msg.sender, pair, liquidity);
        (amountA, amountB) = UniswapV2Pair(pair).burn(to);
        require(amountA >= amountAMin && amountB >= amountBMin, "UniswapV2Router: INSUFFICIENT_OUTPUT_AMOUNT");
    }

    // **** SWAP ****
    function swapExactTokensForTokens(
        uint256 amountIn,
        uint256 amountOutMin,
        address[] calldata path,
        address to,
        uint256 deadline
    ) external returns (uint256[] memory amounts) {
        require(block.timestamp <= deadline, "UniswapV2Router: EXPIRED");
        amounts = getAmountsOut(amountIn, path);
        require(amounts[amounts.length - 1] >= amountOutMin, "UniswapV2Router: INSUFFICIENT_OUTPUT_AMOUNT");
        IERC20(path[0]).safeTransferFrom(msg.sender, UniswapV2Factory(factory).getPair(path[0], path[1]), amounts[0]);
        _swap(amounts, path, to);
    }

    function _swap(uint256[] memory amounts, address[] memory path, address _to) internal {
        for (uint256 i; i < path.length - 1; i++) {
            (address input, address output) = (path[i], path[i + 1]);
            (address token0,) = input < output ? (input, output) : (output, input);
            uint256 amountOut = amounts[i + 1];
            (uint256 amount0Out, uint256 amount1Out) = input == token0 ? (uint256(0), amountOut) : (amountOut, uint256(0));
            address to = i < path.length - 2 ? UniswapV2Factory(factory).getPair(output, path[i + 2]) : _to;
            UniswapV2Pair(UniswapV2Factory(factory).getPair(input, output)).swap(amount0Out, amount1Out, to, "");
        }
    }

    // **** LIBRARY FUNCTIONS ****
    function quote(uint256 amountA, uint256 reserveA, uint256 reserveB) public pure returns (uint256 amountB) {
        require(amountA > 0, "UniswapV2Router: INSUFFICIENT_AMOUNT");
        require(reserveA > 0 && reserveB > 0, "UniswapV2Router: INSUFFICIENT_LIQUIDITY");
        amountB = amountA * reserveB / reserveA;
    }

    function getAmountOut(uint256 amountIn, uint256 reserveIn, uint256 reserveOut) public pure returns (uint256 amountOut) {
        require(amountIn > 0, "UniswapV2Router: INSUFFICIENT_INPUT_AMOUNT");
        require(reserveIn > 0 && reserveOut > 0, "UniswapV2Router: INSUFFICIENT_LIQUIDITY");
        uint256 amountInWithFee = amountIn * 997;
        uint256 numerator = amountInWithFee * reserveOut;
        uint256 denominator = reserveIn * 1000 + amountInWithFee;
        amountOut = numerator / denominator;
    }

    function getAmountsOut(uint256 amountIn, address[] memory path) public view returns (uint256[] memory amounts) {
        require(path.length >= 2, "UniswapV2Router: INVALID_PATH");
        amounts = new uint256[](path.length);
        amounts[0] = amountIn;
        for (uint256 i; i < path.length - 1; i++) {
            (uint112 reserveIn, uint112 reserveOut,) = UniswapV2Pair(UniswapV2Factory(factory).getPair(path[i], path[i + 1])).getReserves();
            amounts[i + 1] = getAmountOut(amounts[i], reserveIn, reserveOut);
        }
    }
}
```

The router:
- Handles adding liquidity with optimal amounts.
- Removes liquidity and returns tokens.
- Swaps tokens along a path (supports multi-hop trades).
- Includes helper functions like `quote`, `getAmountOut`, and `getAmountsOut`.

### 35.2.4 Fee Mechanism

The fee is 0.3% (3/1000) as seen in the `swap` function: `balance0Adjusted = balance0 * 1000 - amount0In * 3`. The fee is taken from the input amount and added to the pool, increasing the constant product `k` slightly, which benefits liquidity providers.

---

## 35.3 Frontend Development

We'll build a simple React frontend that allows users to:
- Select tokens (from a hardcoded list for simplicity).
- Add liquidity.
- Swap tokens.

**Components:**
- `AddLiquidity`: Form to input amounts of two tokens and add liquidity.
- `Swap`: Form to swap tokens.
- `Pool`: Display user's LP token balance and allow removal.

We'll use ethers.js for blockchain interaction.

**Key code snippets:**

```javascript
// utils/contracts.js
import { ethers } from 'ethers';
import FactoryABI from './abis/Factory.json';
import PairABI from './abis/Pair.json';
import RouterABI from './abis/Router.json';
import ERC20ABI from './abis/ERC20.json';

export const factoryAddress = '0x...';
export const routerAddress = '0x...';

export function getFactory(signer) {
  return new ethers.Contract(factoryAddress, FactoryABI, signer);
}

export function getRouter(signer) {
  return new ethers.Contract(routerAddress, RouterABI, signer);
}

export function getPair(address, signer) {
  return new ethers.Contract(address, PairABI, signer);
}

export function getToken(address, signer) {
  return new ethers.Contract(address, ERC20ABI, signer);
}
```

**Add Liquidity Component:**
```javascript
import { useState } from 'react';
import { useWeb3 } from '../context/Web3Context';
import { getRouter, getToken } from '../utils/contracts';
import { ethers } from 'ethers';

export default function AddLiquidity({ tokenA, tokenB }) {
  const { signer, account } = useWeb3();
  const [amountA, setAmountA] = useState('');
  const [amountB, setAmountB] = useState('');
  const [loading, setLoading] = useState(false);

  const handleAddLiquidity = async () => {
    if (!signer || !account) return;
    setLoading(true);
    try {
      const router = getRouter(signer);
      const tokenAContract = getToken(tokenA.address, signer);
      const tokenBContract = getToken(tokenB.address, signer);

      // Approve router to spend tokens
      const amountAWei = ethers.parseUnits(amountA, tokenA.decimals);
      const amountBWei = ethers.parseUnits(amountB, tokenB.decimals);

      await tokenAContract.approve(routerAddress, amountAWei);
      await tokenBContract.approve(routerAddress, amountBWei);

      // Add liquidity
      const deadline = Math.floor(Date.now() / 1000) + 60 * 20; // 20 minutes
      const tx = await router.addLiquidity(
        tokenA.address,
        tokenB.address,
        amountAWei,
        amountBWei,
        0, // slippage tolerance: accept any amount
        0,
        account,
        deadline
      );
      await tx.wait();
      alert('Liquidity added successfully');
    } catch (error) {
      console.error(error);
      alert('Failed to add liquidity');
    } finally {
      setLoading(false);
    }
  };

  return (
    <div>
      <h2>Add Liquidity</h2>
      <input
        type="number"
        placeholder={`Amount ${tokenA.symbol}`}
        value={amountA}
        onChange={(e) => setAmountA(e.target.value)}
      />
      <input
        type="number"
        placeholder={`Amount ${tokenB.symbol}`}
        value={amountB}
        onChange={(e) => setAmountB(e.target.value)}
      />
      <button onClick={handleAddLiquidity} disabled={loading}>
        {loading ? 'Adding...' : 'Add Liquidity'}
      </button>
    </div>
  );
}
```

**Swap Component:**
```javascript
import { useState } from 'react';
import { useWeb3 } from '../context/Web3Context';
import { getRouter, getToken } from '../utils/contracts';
import { ethers } from 'ethers';

export default function Swap({ tokenIn, tokenOut }) {
  const { signer, account } = useWeb3();
  const [amountIn, setAmountIn] = useState('');
  const [amountOut, setAmountOut] = useState('');
  const [loading, setLoading] = useState(false);

  const handleSwap = async () => {
    if (!signer || !account) return;
    setLoading(true);
    try {
      const router = getRouter(signer);
      const tokenInContract = getToken(tokenIn.address, signer);

      const amountInWei = ethers.parseUnits(amountIn, tokenIn.decimals);

      // Approve router
      await tokenInContract.approve(routerAddress, amountInWei);

      // Swap
      const deadline = Math.floor(Date.now() / 1000) + 60 * 20;
      const path = [tokenIn.address, tokenOut.address];
      const tx = await router.swapExactTokensForTokens(
        amountInWei,
        0, // accept any amount out (in production, set min)
        path,
        account,
        deadline
      );
      await tx.wait();
      alert('Swap successful');
    } catch (error) {
      console.error(error);
      alert('Swap failed');
    } finally {
      setLoading(false);
    }
  };

  return (
    <div>
      <h2>Swap</h2>
      <input
        type="number"
        placeholder={`Amount ${tokenIn.symbol}`}
        value={amountIn}
        onChange={(e) => setAmountIn(e.target.value)}
      />
      <p>≈ {amountOut} {tokenOut.symbol}</p>
      <button onClick={handleSwap} disabled={loading}>
        {loading ? 'Swapping...' : 'Swap'}
      </button>
    </div>
  );
}
```

---

## 35.4 Testing and Security

### 35.4.1 Comprehensive Testing

We need to test the core functionality:

- **Creating pairs**: Factory should deploy pair with correct tokens.
- **Adding liquidity**: LP tokens minted correctly, reserves updated.
- **Removing liquidity**: LP tokens burned, tokens returned.
- **Swapping**: Price calculation, fee deduction, event emission.
- **Edge cases**: Zero liquidity, large swaps, flash loans (if implemented).

**Hardhat test example:**
```javascript
const { expect } = require("chai");
const { ethers } = require("hardhat");

describe("UniswapV2", function () {
  let factory, router, token0, token1;
  let owner, addr1;

  beforeEach(async function () {
    [owner, addr1] = await ethers.getSigners();

    // Deploy tokens
    const Token = await ethers.getContractFactory("ERC20Mock");
    token0 = await Token.deploy("Token0", "TK0", 18);
    token1 = await Token.deploy("Token1", "TK1", 18);

    // Deploy factory
    const Factory = await ethers.getContractFactory("UniswapV2Factory");
    factory = await Factory.deploy();

    // Deploy router
    const Router = await ethers.getContractFactory("UniswapV2Router");
    router = await Router.deploy(factory.address);
  });

  describe("Factory", function () {
    it("Should create a pair", async function () {
      await factory.createPair(token0.address, token1.address);
      const pairAddress = await factory.getPair(token0.address, token1.address);
      expect(pairAddress).to.not.equal(ethers.constants.AddressZero);
    });
  });

  describe("Liquidity", function () {
    beforeEach(async function () {
      await factory.createPair(token0.address, token1.address);
      pairAddress = await factory.getPair(token0.address, token1.address);
      pair = await ethers.getContractAt("UniswapV2Pair", pairAddress);
    });

    it("Should add liquidity", async function () {
      // Mint tokens to owner
      await token0.mint(owner.address, ethers.parseEther("1000"));
      await token1.mint(owner.address, ethers.parseEther("1000"));

      // Approve router
      await token0.approve(router.address, ethers.parseEther("100"));
      await token1.approve(router.address, ethers.parseEther("100"));

      // Add liquidity
      const deadline = Math.floor(Date.now() / 1000) + 3600;
      await router.addLiquidity(
        token0.address,
        token1.address,
        ethers.parseEther("100"),
        ethers.parseEther("100"),
        0,
        0,
        owner.address,
        deadline
      );

      const reserves = await pair.getReserves();
      expect(reserves[0]).to.equal(ethers.parseEther("100"));
      expect(reserves[1]).to.equal(ethers.parseEther("100"));
    });
  });
});
```

### 35.4.2 Security Considerations

- **Reentrancy**: The pair contract uses the checks-effects-interactions pattern; external calls are at the end of functions.
- **Overflow/underflow**: Solidity 0.8+ has built-in checks.
- **Flash loan callback**: The `swap` function allows arbitrary calls; ensure that callbacks cannot harm the pool (standard Uniswap design is safe).
- **Reserve manipulation**: `skim` and `sync` prevent manipulation by donating tokens.
- **Access control**: Only the factory can initialize a pair.

**Audit:** Before mainnet deployment, consider a professional audit. For learning purposes, thorough testing suffices.

---

## 35.5 Deployment and Launch

Deploy the contracts in order:

1. Deploy `UniswapV2Factory`.
2. Deploy `UniswapV2Router` with the factory address.

**Deployment script (Hardhat):**
```javascript
async function main() {
  const [deployer] = await ethers.getSigners();
  console.log("Deploying with account:", deployer.address);

  // Deploy factory
  const Factory = await ethers.getContractFactory("UniswapV2Factory");
  const factory = await Factory.deploy();
  await factory.waitForDeployment();
  console.log("Factory deployed to:", await factory.getAddress());

  // Deploy router
  const Router = await ethers.getContractFactory("UniswapV2Router");
  const router = await Router.deploy(await factory.getAddress());
  await router.waitForDeployment();
  console.log("Router deployed to:", await router.getAddress());
}

main().catch(console.error);
```

After deployment, verify contracts on Etherscan.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 35 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  We built a complete DEX inspired by Uniswap V2:               │
│    • Factory: creates pairs with deterministic addresses       │
│    • Pair: holds reserves, mints LP tokens, swaps with 0.3% fee│
│    • Router: convenience functions for users                   │
│                                                                 │
│  Frontend: React app with AddLiquidity and Swap components     │
│  Testing: Hardhat tests cover core functionality               │
│  Security: checks-effects-interactions, overflow protection    │
│                                                                 │
│  This project demonstrates real-world DeFi engineering.        │
│  Extend it with TWAP oracles, farming, or governance.          │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 36 – Building an NFT Marketplace. We'll create a marketplace for ERC-721 tokens with listing, purchasing, auctions, and royalties, integrating IPFS for metadata.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../9. production_and_deployment/34. legal_and_regulatory_considerations.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='36. building_an_nft_marketplace.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
